# Python Data Model

Most of the content of this book has been extracted from the book "Fluent Python" by Luciano Ramalho (O'Reilly, 2015)
http://shop.oreilly.com/product/0636920032519.do

>One of the best qualities of Python is its consistency. 
>After working with Python for a while, you are able to start making informed, correct guesses about features that are new to you.

However, if you learned another object-oriented language before Python, you may have found it strange to use `len(collection)` instead of `collection.len()`. 

This apparent oddity is the tip of an iceberg that, when properly understood, is the key to everything we call **Pythonic**. 

>The iceberg is called the **Python data model**, and it describes the API that you can use to make your own objects play well with the most idiomatic language features.

> You can think of the data model as a description of Python as a framework. It formalises the interfaces of the building blocks of the language itself, such as sequences, iterators, functions, classes, context managers, and so on.

While coding with any framework, you spend a lot of time implementing methods that are called by the framework. The same happens when you leverage the Python data model. 

The Python interpreter invokes **special methods** to perform basic object operations, often triggered by **special syntax**. 

The special method names are always written with leading and trailing double underscores (i.e., `__getitem__`).

For example, the syntax `obj[key]` is supported by the `__getitem__` special method. 

In order to evaluate `my_collection[key]`, the interpreter calls `my_collection.__getitem__(key)`.

The special method names allow your objects to implement, support, and interact with basic language constructs such as:

- Iteration
- Collections
- Attribute access
- Operator overloading
- Function and method invocation
- Object creation and destruction
- String representation and formatting
- Managed contexts (i.e., with blocks)

## A Pythonic Card Deck

In [11]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

The first thing to note is the use of `collections.namedtuple` to construct a simple class to represent individual cards. 

Since Python 2.6, `namedtuple` can be used to build classes of objects that are just bundles of attributes with no custom methods, like a database record. 

In the example, we use it to provide a nice representation for the cards in the deck:

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

But the point of this example is the `FrenchDeck` class. 

It’s short, but it packs a punch. 

##### Length of a Deck

First, like any standard Python collection, a deck responds to the `len()` function by returning the number of cards in it:

In [4]:
deck = FrenchDeck()
len(deck)

52

##### Reading Specific Cards

Reading specific cards from the deck say, the first or the last— should be as easy as `deck[0]` or `deck[-1]`, and this is what the `__getitem__` method provides:

In [5]:
deck[0]

Card(rank='2', suit='spades')

In [6]:
deck[-1]

Card(rank='A', suit='hearts')

##### Picking Random Card

Should we create a method to pick a random card? **No need**. 

Python already has a function to get a random item from a sequence: `random.choice`. We can just use it on a deck instance:

In [7]:
from random import choice
choice(deck)

Card(rank='Q', suit='clubs')

In [8]:
choice(deck)

Card(rank='A', suit='spades')

In [9]:
choice(deck)

Card(rank='K', suit='spades')

### First Impressions:

We’ve just seen two advantages of using special methods to leverage the Python data model:

- The users of your classes don’t have to memorize arbitrary method names for standard operations 
(“How to get the number of items? Is it .size(), .length(), or what?”).

- It’s easier to benefit from the rich Python standard library and avoid reinventing the wheel, like the `random.choice` function.

**... but it gets better ...**

Because our `__getitem__` delegates to the `[]` operator of `self._cards`, our deck automatically supports **slicing**. 

Here’s how we look at the top three cards from a brand new deck, and then pick just the aces by starting on index 12 and skipping 13 cards at a time:



In [10]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [11]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

Just by implementing the `__getitem__` special method, our deck is also **iterable**:


In [12]:
for card in deck:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

... also in **reverse order**:

```python 
for card in reversed(deck):
    print(card)
    
...
```

## To know more...

To have a more complete overview of the Magic (_dunder_) methods, please have a look at the [Extra - Magic Methods and Operator Overloading](Extra - Magic Methods and Operator Overloading.ipynb) notebook.

---

## Exercise: Emulating Numeric Types

```python 
from math import hypot

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
```